<a href="https://www.kaggle.com/code/archismancoder/fb-bartforcondgen-lv3-batch8-hftrainer?scriptVersionId=226105455" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install transformers pandas openpyxl torch peft bs4 emoji joblib scikit-learn ray huggingface_hub datasets optuna lxml optim

  Preparing metadata (setup.py) ... done
  Created wheel for optim: filename=optim-0.1.0-py2.py3-none-any.whl size=2707 sha256=03b837ad7d5eac32e3d709cd42029af05c5aebbdc40b07cc6ea691604e684469
  Stored in directory: /root/.cache/pip/wheels/63/cd/16/e7762fdd7862a4f618fa7ca62119fac2112de90041cee77227
Successfully built optim


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tachygraphy-dataset/tweet_emotion_dataset.csv
/kaggle/input/tachygraphy-dataset/output-2.xlsx
/kaggle/input/tachygraphy-dataset/Tachygraphy_MicroText-AIO-V3.xlsx
/kaggle/input/tachygraphy-dataset/Tachygraphy_MicroText-AIO-Emotion_Mood_Tags-V3.xlsx
/kaggle/input/tachygraphy-dataset/Tachygraphy_dataset_main.csv
/kaggle/input/tachygraphy-dataset/Tachygraphy_MicroText-AIO-Sentiment_Polarities_3_Label-V3.xlsx
/kaggle/input/tachygraphy-dataset/Tachygraphy_MicroText-AIO-Sentiment_Polarities_2_Label-V3 (Neutral Omitted).xlsx


In [3]:
dataset = pd.read_excel('/kaggle/input/dataset-tachygraphy/Tachygraphy_MicroText-AIO-V3.xlsx')

In [4]:
df = dataset

In [5]:
df.rename(columns={'Informal Text':'input', 'Expanded Meaning':'target'}, inplace = True)

In [6]:
df.head()

,input,target
0,"omg, JEE prep is killing me rn","Oh my god, Joint Entrance Examination preparat..."
1,u up 4 a break b4 UPSC revision?,Are you up for a break before Union Public Ser...
2,"ttyl, finishing da CAT mock","Talk to you later, finishing the Common Admiss..."
3,"nah, dat GATE paper was brutal af","No, that Graduate Aptitude Test in Engineering..."
4,sup? u done w/ ur IIT assignment?,What's up? Are you done with your Indian Insti...


In [7]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [8]:
import torch
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BertModel, BertTokenizer
# from torch.cuda.amp import autocast, GradScaler
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import ray
from ray import tune
import torch.nn as nn

from torch.optim import AdamW
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
import optuna
# from optuna.integration import PyTorchLightningPruner
from ray import tune
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import ASHAScheduler
# from ray.tune.integration.pytorch import TuneReportCallback
from torch.amp import GradScaler, autocast
# from ray.tune.integration.optuna import OptunaSearch
from ray.tune.search.optuna import OptunaSearch
from ray import tune
from ray.tune.search.hyperopt import HyperOptSearch
from torch import autocast
# from ray import tune
# from ray.tune.integration.tensorboard import TensorBoardReporter
from ray.tune.logger import TBXLogger
# from torch.utils.tensorboard import SummaryWriter
from ray.train import report
# from ray.tune.integration.jupyter import JupyterNotebookReporter
from ray.tune import JupyterNotebookReporter
# from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from ray.tune.schedulers import HyperBandScheduler, AsyncHyperBandScheduler


import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings


import emoji
from bs4 import BeautifulSoup
import os
import re
import string
import json


import argparse # CPMP

import scipy as sp

from tqdm.auto import tqdm

In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
#     text = emoji.demojize(text)
#     text = re.sub(r'\:(.*?)\:','',text)
#     text = str(text).lower()    #Making Text Lowercase
#     text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, 0-9, "%", ".", "&", ",", "'", "?", "!", ",", "'", ";", "-")
    text = re.sub(r"[^a-zA-Z0-9?.!,¿'%&,';-]+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
#     text = clean_contractions(text, contraction_mapping)
    text = clean_text(text)
#     text = clean_contractions(text, contraction_mapping)
#     text = clean_special_chars(text, punct, punct_mapping)
#     text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [10]:
df['input'] = df['input'].apply(lambda x: text_preprocessing_pipeline(x))
df['target'] = df['target'].apply(lambda x: text_preprocessing_pipeline(x))

<ipython-input-9-93bb6d8e489f>:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'lxml').get_text()


In [11]:
df['input'] = df['input'].astype(str)
df['target'] = df['target'].astype(str)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
import torch
import transformers
from datasets import Dataset
from transformers import LlamaForCausalLM, LlamaTokenizer, TrainingArguments, Trainer, AutoTokenizer, BartForConditionalGeneration

In [14]:
MODEL_NAME = 'facebook/bart-base'

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
def test_train_split(dataset, test_ratio=0.1):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [17]:
train_ds_pd, validation_ds_pd = test_train_split(df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(validation_ds_pd)))

9277 examples in training, 1003 examples in testing.


In [18]:
train_ds_pd = df

In [19]:
def tokenize_function(batch):
    model_inputs = tokenizer(batch["input"], padding="longest", truncation=False)
    labels = tokenizer(batch["target"], padding="longest", truncation=False)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
dataset1 = Dataset.from_pandas(train_ds_pd)
dataset2 = Dataset.from_pandas(validation_ds_pd)

In [21]:
tokenized_dataset = dataset1.map(tokenize_function)
tokenized_dataset2 = dataset2.map(tokenize_function)

Map:   0%|          | 0/10280 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

In [22]:
model = BartForConditionalGeneration.from_pretrained(
    MODEL_NAME,  # BF16 for H100
    device_map=torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Auto GPU allocation
)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [23]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")


In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bart-checkpoints",
    evaluation_strategy="steps",  # Evaluate at regular intervals
    save_strategy="steps",  # Save model checkpoints at regular steps
    logging_strategy="steps",  # Log at regular steps
    logging_dir="/kaggle/working/logs",  # Directory for TensorBoard logs
    logging_steps=50,  # Log loss/metrics every 50 steps
    log_level="debug",
    save_total_limit=2,  # Keep only the 2 latest checkpoints
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=15,
    lr_scheduler_type="linear",
    warmup_steps=50,
    fp16=True,  # Mixed precision for efficiency
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to=["tensorboard"],
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset2,
    tokenizer=tokenizer,
    data_collator=data_collator  # Ensures dynamic padding
)


trainer.train()

<ipython-input-25-abe393457df6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend
Currently training with a batch size of: 8
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input, target. If input, target are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,280
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 19,275
  Number of trainable parameters = 1

Step,Training Loss,Validation Loss
50,1.913700,1.142654
100,1.162400,0.919556
150,0.916000,0.759303
200,0.868000,0.740576
250,0.842900,0.625380
300,0.767400,0.590813
350,0.746900,0.563534
400,0.740500,0.575982
450,0.720600,0.507351
500,0.637800,0.508837


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input, target, __index_level_0__. If input, target, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1003
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input, target, __index_level_0__. If input, target, __index_level_0__ are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1003
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: input, target, __index_level_0__. If input, target, __index_level_0__ are not expected by

TrainOutput(global_step=19275, training_loss=0.14067529849041296, metrics={'train_runtime': 3037.9265, 'train_samples_per_second': 50.758, 'train_steps_per_second': 6.345, 'total_flos': 2497428133724160.0, 'train_loss': 0.14067529849041296, 'epoch': 15.0})

In [26]:
# Save the fine-tuned model
model.save_pretrained("/kaggle/working/bartbase_finetuned")
tokenizer.save_pretrained("/kaggle/working/bartbase_finetuned")
print("Model saved successfully!")

Configuration saved in /kaggle/working/bartbase_finetuned/config.json
Configuration saved in /kaggle/working/bartbase_finetuned/generation_config.json
Model weights saved in /kaggle/working/bartbase_finetuned/model.safetensors
tokenizer config file saved in /kaggle/working/bartbase_finetuned/tokenizer_config.json
Special tokens file saved in /kaggle/working/bartbase_finetuned/special_tokens_map.json


Model saved successfully!


In [27]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HF_READ_TOKEN")
# secret_value_1 = user_secrets.get_secret("HF_READ_WRITE_TOKEN")

In [28]:
from huggingface_hub import login

login(token=user_secrets.get_secret("HF_READ_TOKEN"))

In [ ]:
from huggingface_hub import login

# Replace 'your_hf_token' with your actual Hugging Face API token
login(token=user_secrets.get_secret("HF_READ_WRITE_TOKEN"))

In [29]:
from huggingface_hub import HfApi, create_repo, upload_folder

# Define your Hugging Face repo ID
repo_id = "tachygraphy-microtrext-norm-org/BART-base-HF-Trainer-Batch8"  # Example

# Local directory where all your model files are stored
local_model_dir = "/kaggle/working/bartbase_finetuned"

# Initialize API instance
api = HfApi()

# Create the repo if it doesn't exist
create_repo(repo_id, exist_ok=True)

# Upload all the files from the local directory to the Hugging Face Hub
upload_folder(
    folder_path=local_model_dir,  # Path to the folder to upload
    repo_id=repo_id,              # Repo ID on Hugging Face
    repo_type="model"             # Specify it's a model repo
)

print(f"All files uploaded to Hugging Face under {repo_id}")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

All files uploaded to Hugging Face under tachygraphy-microtrext-norm-org/BART-base-HF-Trainer-Batch8


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
def predict(model, tokenizer, input_texts, device):
#     model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
#     model.load_state_dict(torch.load(model_path)['model_state_dict'])
    model.to(device)
#     model.eval()

    predictions = []
    with torch.no_grad():
#         for text in input_texts:
        text = input_texts
        padded = tokenizer(text, return_tensors='pt', truncation=False, padding="longest").to(device)
        
        input_ids = padded['input_ids'].to(device)
        attention_mask = padded['attention_mask'].to(device)

        # max_new_tokens = min(input_ids.shape[1] * 2, 1024)
        max_new_tokens = 1024 # default since we are not sure about the word expansions, like brb -> be right back there can short form
                              # of very large length words
            
        preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens, temperature=0.4, do_sample=True, top_k=50)
        # preds = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_new_tokens)
        pred_text = tokenizer.decode(preds[0], skip_special_tokens=True)
        predictions.append(pred_text)
    
    return predictions

In [32]:
# Test prediction on random input
random_input = "thx 4 the tip. will let u no. btw gotta use smpl things 4 now. tbh i'm tired, iykyk"
predicted_text = predict(model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: thx 4 the tip. will let u no. btw gotta use smpl things 4 now. tbh i'm tired, iykyk
Predicted Meaning: ["Thanks for the tip. I'll let you no. By the way I have to use some things for now. To be honest I'm tired, I don't know."]


In [33]:
# Test prediction on random input
random_input = "pls hlp, idk wht 2 do"
predicted_text = predict(model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: pls hlp, idk wht 2 do
Predicted Meaning: ["please help I don't know what to do"]


In [34]:
# Test prediction on random input
random_input = "i don8 no fr y hes soooo sad"
predicted_text = predict(model, tokenizer, random_input, device)
print(f"Input: {random_input}")
print(f"Predicted Meaning: {predicted_text}")

Input: i don8 no fr y hes soooo sad
Predicted Meaning: ["i don't care no fr why he is soooo sad"]


The output sentences might vary, try to generate 2-3 samples and take the best one. Below one is a good example where this model might get confused.

In [35]:
!zip -r ./logs.zip /kaggle/working/logs



  adding: kaggle/working/logs/ (stored 0%)
  adding: kaggle/working/logs/events.out.tfevents.1741185119.1533a7b35ef5.18.0 (deflated 71%)
